In [10]:
import sys
from pathlib import Path

# Go from notebooks/ up to project root
project_root = Path("..").resolve()

# Point at the src/ folder
src_path = project_root / "src"

# Add src/ to sys.path so we can do `from dcf_model...`
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

print("Project root:", project_root)
print("Src path:", src_path)


Project root: /Users/norajohannessen/Git_ptojects/dcf/dcf-valuation
Src path: /Users/norajohannessen/Git_ptojects/dcf/dcf-valuation/src


In [14]:
from dcf_model.assumptions import WACCInputs

def format_wacc_case(label: str, wacc_inputs: WACCInputs, terminal_growth: float):
    ke = wacc_inputs.risk_free_rate + wacc_inputs.beta * wacc_inputs.equity_risk_premium
    kd_after_tax = wacc_inputs.cost_of_debt * (1 - wacc_inputs.tax_rate)
    wacc = wacc_inputs.equity_weight * ke + wacc_inputs.debt_weight * kd_after_tax

    return {
        "Scenario": label,
        "Risk-free rate": f"{wacc_inputs.risk_free_rate:.2%}",
        "Beta": round(wacc_inputs.beta, 3),
        "Equity risk premium": f"{wacc_inputs.equity_risk_premium:.2%}",
        "Cost of equity": f"{ke:.2%}",
        "Cost of debt (pre-tax)": f"{wacc_inputs.cost_of_debt:.2%}",
        "Tax rate": f"{wacc_inputs.tax_rate:.2%}",
        "Cost of debt (after tax)": f"{kd_after_tax:.2%}",
        "Equity weight": f"{wacc_inputs.equity_weight:.0%}",
        "Debt weight": f"{wacc_inputs.debt_weight:.0%}",
        "WACC": f"{wacc:.2%}",
        "Terminal growth": f"{terminal_growth:.2%}",
    }


In [ ]:
import pandas as pd

from dcf_model.data_fetcher import build_company_financials_from_yahoo
from dcf_model.auto_metrics import compute_historical_metrics
from dcf_model.heuristics import build_assumption_suggestions
from dcf_model.assumptions import WACCInputs

df_hist, fetch_diag = build_company_financials_from_yahoo("APPL")
metrics, quality = compute_historical_metrics(df_hist)

# ---- Bear case ----
wacc_bear = WACCInputs(
    risk_free_rate=0.042,
    beta=1.20,
    equity_risk_premium=0.050,
    cost_of_debt=0.050,
    tax_rate=0.21,
    equity_weight=0.60,
    debt_weight=0.40,
)

# ---- Base case ----
wacc_base = WACCInputs(
    risk_free_rate=0.042,
    beta=1.10,
    equity_risk_premium=0.050,
    cost_of_debt=0.045,
    tax_rate=0.21,
    equity_weight=0.70,
    debt_weight=0.30,
)

# ---- Bull case ----
wacc_bull = WACCInputs(
    risk_free_rate=0.042,
    beta=0.95,
    equity_risk_premium=0.050,
    cost_of_debt=0.040,
    tax_rate=0.21,
    equity_weight=0.80,
    debt_weight=0.20,
)

suggestions = build_assumption_suggestions(
    metrics,
    quality,
    years=5,
    wacc_inputs_bear=wacc_bear,
    wacc_inputs_base=wacc_base,
    wacc_inputs_bull=wacc_bull,
    terminal_growth_bear=0.015,
    terminal_growth_base=0.02,
    terminal_growth_bull=0.025,
    fallback_growth=None,   # or explicitly set if you want
    fallback_margin=None,   # same here
)

rows = [
    format_wacc_case("Bear", wacc_bear, 0.015),
    format_wacc_case("Base", wacc_base, 0.020),
    format_wacc_case("Bull", wacc_bull, 0.025),
]

wacc_table = pd.DataFrame(rows)
display(wacc_table)

print("\n--- FETCH DIAGNOSTICS ---")
print(fetch_diag)

print("\n--- DATA QUALITY REPORT ---")
print(f"Original rows: {quality.n_original_rows}")
print(f"Rows after cleaning: {quality.n_rows_after_cleaning}")
print(f"Dropped years: {quality.dropped_years}")
print("Notes:")
for note in quality.notes:
    print(" -", note)

print("\n--- ASSUMPTION WARNINGS ---")
if suggestions.warnings.messages:
    for msg in suggestions.warnings.messages:
        print(" -", msg)
else:
    print("None ✅")




ValueError: EQNR.OL appears to be a financial institution (bank/insurer). EBIT-based DCF is not appropriate. Use a bank-specific valuation model.